In [1]:
from keras.models import load_model

Using TensorFlow backend.


In [2]:
import cv2

In [3]:
import numpy as np

In [4]:
x=160
y=25
h=370
w=345
clr = (0, 255, 0)
brd = 2

y_0 = y+brd
y_1 = y+h-brd-1
x_0 = x+brd
x_1 = x+w-brd-1

IMG_ROWS = 48
IMG_COLS = 48

In [5]:
classes_dict = {
    0: "book",
    1: "film",
    2: "show",
    3: "geo",
    4: "phrase",
    5: "series",
    6: "one",
    7: "two",
    8: "three",
    9: "four",
    10: "five",
    11: "six",
    12: "seven",
    13: "eight",
    14: "nine",
    15: "ten"
}

In [6]:
img_count = 0

In [7]:
THE_CHOSEN_MODEL = "Charades-CNN-48x3-ThreeDropout-Batch-32-l1-0x001"

In [8]:
saved_model = load_model(THE_CHOSEN_MODEL + '.model')

In [9]:
def preprocess(img):
    # izecanje regiona od znacaja
    crop_img = img[y_0:y_1, x_0:x_1]
    # otkljanjanje suma -> gausovsko zamucivanje
    blur_img = cv2.GaussianBlur(crop_img, (5,5), 0)
    # dodavanje maske u tonu koze -> hsv (dobar za izdvajanje objekata odredjene boje sa slike)
    # konverzija u hsv prstor boja
    hsv_img = cv2.cvtColor(blur_img, cv2.COLOR_BGR2HSV)
    # odredjivanje donje i gornje granice boje pod koje spada boja koze
    lower_hsv = (0, 7, 55)
    upper_hsv = (15, 190, 255)
    # tresholding pomocu inRange
    hsv_mask = cv2.inRange(hsv_img, lower_hsv, upper_hsv)
    # odstranjujemo sum
    hsv_blur_mask = cv2.medianBlur(hsv_mask, 3)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    hsv_ero = cv2.erode(hsv_blur_mask, kernel, iterations=1)
    hsv_open = cv2.dilate(hsv_ero, kernel, iterations=4)
    # konture
    contours, hierarchy = cv2.findContours(hsv_open, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    approved_contours = []
    for contour in contours:
        center, size, angle = cv2.minAreaRect(contour)
        width, height = size
        if width > 60 and height > 50:
            approved_contours.append(contour)
    # obradjena slika
    height, width, channels = crop_img.shape
    black = np.zeros((height, width))

    cv2.fillPoly(black, approved_contours, color=(255,255,255))
    # povratna vrednost -> ulaz za mrezu
    cv2.imshow("preprocesed", black)
    cv2.imwrite("pre/pre_"+str(img_count)+".png", black)
    return black

In [10]:
def charades(frame):
    p_frame = preprocess(frame)
    image = p_frame / 255.0
    image = cv2.resize(image, (IMG_ROWS, IMG_COLS))
    image = image.flatten()
    img_arr = np.array(image).reshape(-1, IMG_ROWS, IMG_COLS, 1)
    
    '''
    prediction_proba = saved_model.predict_proba(img_arr)
    best_pred_value = np.amax(prediction_proba) * 100
    print("value", best_pred_value)
    best_pred = np.argmax(prediction_proba)
    print("best prediction", classes_dict[best_pred])
    '''
    
    prediction = saved_model.predict_classes(img_arr)
    pred_class = classes_dict[prediction[0]]
    return pred_class

In [32]:
# load images from camera

cam = cv2.VideoCapture(0)

cv2.namedWindow("Play charades!")

img_count = 0
charades_class = ""

while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break

    #cv2.rectangle(frame, (x,y), (x+w,y+h), clr, brd)

    cv2.putText(frame,  
                "Let's play some charades",  
                (120, 50),  
                cv2.FONT_HERSHEY_SIMPLEX, 1,  
                (0, 255, 255), 
                1, 
                cv2.LINE_AA)
    
    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed
        img_count += 1
        charades_class = charades(frame)
        print(charades_class)
        
    cv2.putText(frame,  
            "Found: " + charades_class,  
            (50, 400),  
            cv2.FONT_HERSHEY_SIMPLEX, 1,  
            (0, 255, 255), 
            1, 
            cv2.LINE_AA)
    
    cv2.imshow("Play charades!", frame)

cam.release()

cv2.destroyAllWindows()

Escape hit, closing...
